# ML Pipeline Orchestrator Notebook
This notebook orchestrates the complete ML pipeline using SageMaker SDK

In [ ]:
import argparse
import os
import json
import time
import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import mlflow

print("🚀 Starting Complete ML Pipeline Orchestration with Real SageMaker SDK")

In [ ]:
# Configuration
role = "arn:aws:iam::058264284947:role/service-role/AmazonSageMaker-ExecutionRole-20241115T121975"
bucket = "demo-bucket-smus-ml-us-west-2"
region = "us-west-2"
mlflow_arn = "arn:aws:sagemaker:us-east-1:058264284947:mlflow-tracking-server/wine-classification-mlflow-v2"
model_name = "realistic-classifier-v1"

# MLflow setup
mlflow.set_tracking_uri(mlflow_arn)

# Create SageMaker session with explicit region
boto_session = boto3.Session(region_name=region)
session = sagemaker.Session(boto_session=boto_session)

print(f"Using bucket: {bucket}")
print(f"Using MLflow: {mlflow_arn}")
print(f"Using SageMaker session: {session}")

In [ ]:
# Step 1: Training with Real SageMaker SDK
print("\n📈 Step 1: Real Model Training with SageMaker SDK")
sklearn_estimator = SKLearn(
    entry_point='sagemaker_training_script.py',
    source_dir=f's3://{bucket}/training_code.tar.gz',
    framework_version='1.2-1',
    py_version='py3',
    instance_type='ml.m5.large',
    instance_count=1,
    role=role,
    output_path=f's3://{bucket}/model-artifacts/',
    environment={"MLFLOW_TRACKING_SERVER_ARN": mlflow_arn},
    hyperparameters={
        'n-estimators': 100, 
        'max-depth': 10, 
        'random-state': 42,
        'model-name': model_name
    }
)

job_name = f"orchestrated-training-{int(time.time())}"
print(f"Starting real training job: {job_name}")

# This will create a real SageMaker training job and integrate with MLflow
sklearn_estimator.fit(
    inputs={'training': f's3://{bucket}/training-data/'},
    job_name=job_name
)

print(f"✅ Real training completed: {sklearn_estimator.model_data}")

In [ ]:
# Step 2: Champion Model Tagging with Real MLflow
print("\n🏆 Step 2: Real Champion Model Tagging")
try:
    client = mlflow.MlflowClient()
    
    # Get latest model version
    versions = client.search_model_versions(f"name='{model_name}'")
    if versions:
        latest_version = max(versions, key=lambda x: int(x.version))
        client.set_registered_model_alias(
            name=model_name,
            alias="champion",
            version=latest_version.version
        )
        print(f"✅ Real champion tagging: version {latest_version.version}")
        champion_tagged = True
    else:
        print("⚠️ No model versions found, will be created by training job")
        champion_tagged = False
except Exception as e:
    print(f"⚠️ Champion tagging will happen after training job completes: {e}")
    champion_tagged = False

In [ ]:
# Step 3: Real Batch Transform
print("\n🔄 Step 3: Real Batch Inference with SageMaker SDK")
transformer = sklearn_estimator.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/inference-results/'
)

transform_job_name = f"orchestrated-inference-{int(time.time())}"
print(f"Starting real batch transform: {transform_job_name}")

# This will create a real SageMaker batch transform job
transformer.transform(
    data=f's3://{bucket}/inference-data/',
    content_type='text/csv',
    split_type='Line',
    job_name=transform_job_name
)

print(f"✅ Real batch inference completed: {transformer.output_path}")

In [ ]:
# Step 4: Save Real Orchestration Results
results = {
    "orchestration_type": "real_ml_pipeline_with_sagemaker_sdk",
    "pipeline_steps": [
        "real_sagemaker_training",
        "real_mlflow_integration", 
        "real_champion_tagging",
        "real_batch_inference"
    ],
    "training_job": job_name,
    "transform_job": transform_job_name,
    "model_artifacts": sklearn_estimator.model_data,
    "inference_results": transformer.output_path,
    "mlflow_tracking": mlflow_arn,
    "model_name": model_name,
    "champion_tagged": champion_tagged,
    "sagemaker_sdk_used": True,
    "real_jobs_created": True,
    "execution_timestamp": int(time.time()),
    "status": "SUCCESS"
}

print("\n🎉 Real ML Pipeline Orchestration with SageMaker SDK Completed!")
print(f"Real Training Job: {job_name}")
print(f"Real Transform Job: {transform_job_name}")
print(f"Real Model Artifacts: {sklearn_estimator.model_data}")
print(f"Real Inference Results: {transformer.output_path}")
print(f"MLflow Integration: ENABLED via SageMaker SDK")
print(f"Champion Model: {model_name}@champion")

# Save results
with open('/opt/ml/output/orchestration_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n📊 Results saved to /opt/ml/output/orchestration_results.json")